#### This notebook demonstrates the use of the EOP post-processing algorithm for bias mitigation.


In [31]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from eq_odds_postprocessing import EqOddsPostprocessing
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle
from sklearn.linear_model import Lasso
from random import sample

## Huangrui's Dataset 

In [32]:
class GermanDataset(StandardDataset):
    """German credit Dataset.

    See :file:`aif360/data/raw/german/README.md`.
    """

    def __init__(self, path, label_name='Y', favorable_classes=[1],
                 protected_attribute_names=['sex', 'age'],
                 privileged_classes=[[1],[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[], features_to_drop=[],
                 na_values=[], custom_preprocessing=None,
                 metadata=None):
        
        df = pd.read_csv(path)
        

        super(GermanDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

#### Load dataset and specify options

In [33]:
## import dataset
dataset_used = "german" # "german", "german", "compas"
protected_attribute_used = 2 # 1, 2


#     dataset_orig = GermanDataset()
if protected_attribute_used == 1:
    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]
else:
    privileged_groups = [{'age': 1}]
    unprivileged_groups = [{'age': 0}]
        
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
        
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

#### Split into train, test and validation

In [34]:
experiments_info = {}
budget = 1
for K in range(1, 6):
    if protected_attribute_used == 1:
        dataset_orig_train= GermanDataset(path="./Huangrui/german/german_train{}.csv".format(K),protected_attribute_names=['sex'],
                    privileged_classes=[[1]])
        dataset_orig_test= GermanDataset(path="./Huangrui/german/german_test{}.csv".format(K),protected_attribute_names=['sex'],
                    privileged_classes=[[1]])
    else:
        dataset_orig_train= GermanDataset(path="./Huangrui/german/german_train{}.csv".format(K),protected_attribute_names=['age'],
                    privileged_classes=[[1]])
        dataset_orig_test= GermanDataset(path="./Huangrui/german/german_test{}.csv".format(K),protected_attribute_names=['age'],
                    privileged_classes=[[1]])
    #only use the budget% of the training data
    dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)
    # Lasso linear classifier and predictions
    scale_orig = StandardScaler()
    X_train = scale_orig.fit_transform(dataset_orig_train.features)
    y_train = dataset_orig_train.labels.ravel()
    if protected_attribute_used == 1:
        lmod = pickle.load(open('experiments/german'+str(K)+'_sex_bmodel.pkl','rb'))["clf"]
    else:
        lmod = pickle.load(open('experiments/german'+str(K)+'_age_bmodel.pkl','rb'))["clf"]
    y_train_pred = lmod.predict(X_train)

    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.labels = y_train_pred>0.5
    sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
    dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)
    #get the biased predictions for test
    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = scale_orig.transform(dataset_orig_test_pred.features)
    y_test = dataset_orig_test_pred.labels
    dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)
    #load EOP Model
    EOP = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=random_seed)
    #fit EOP model
    EOP = EOP.fit(dataset_orig_train, dataset_orig_train_pred)
    print("Fitted the EOP model")
    # get the EOP predictions for test (Transform the test set)
    dataset_transf_test_pred = EOP.predict(dataset_orig_test_pred)

    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                    unprivileged_groups, privileged_groups)

    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}

Fitted the EOP model
Balanced accuracy = 0.9370
Statistical parity difference = 0.0915
Disparate impact = 1.1389
Average odds difference = 0.1446
Equal opportunity difference = 0.1074
Theil index = 0.0712
K = 1, budget = 1
The Error for the test dataset is 0.075
The Equal opportunity difference for the test dataset is 0.1074
Fitted the EOP model
Balanced accuracy = 0.9485
Statistical parity difference = -0.0383
Disparate impact = 0.9379
Average odds difference = 0.0614
Equal opportunity difference = 0.1228
Theil index = 0.0726
K = 2, budget = 1
The Error for the test dataset is 0.07
The Equal opportunity difference for the test dataset is 0.1228
Fitted the EOP model
Balanced accuracy = 0.8973
Statistical parity difference = 0.0794
Disparate impact = 1.1221
Average odds difference = 0.2189
Equal opportunity difference = 0.1520
Theil index = 0.1079
K = 3, budget = 1
The Error for the test dataset is 0.115
The Equal opportunity difference for the test dataset is 0.152
Fitted the EOP model

In [35]:
experiments_info

{'K = 1, budget = 1': {'Error': 0.075,
  'Equal opportunity difference': 0.1074380165289256},
 'K = 2, budget = 1': {'Error': 0.07,
  'Equal opportunity difference': 0.1228070175438597},
 'K = 3, budget = 1': {'Error': 0.115,
  'Equal opportunity difference': 0.15200000000000002},
 'K = 4, budget = 1': {'Error': 0.075,
  'Equal opportunity difference': 0.11111111111111116},
 'K = 5, budget = 1': {'Error': 0.1,
  'Equal opportunity difference': 0.15929203539823011}}